In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
# Load API Credentials
with open('/Users/svaze/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)




In [3]:
# set our API call parameters and filename before the first call
LOCATION = 'San Francisco, CA,94133'
TERM = 'Italian'




In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Italian.json"
JSON_FILE




'Data/results_in_progress_Italian.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")



[i] Data/results_in_progress_Italian.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results



457

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



23

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [13]:
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/23 [00:00<?, ?it/s]

In [14]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)



  0%|          | 0/23 [00:00<?, ?it/s]

In [15]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8dUaybEPHsZMgr1iKgqgMQ,sotto-mare-san-francisco-4,Sotto Mare,https://s3-media3.fl.yelpcdn.com/bphoto/o3hIcG...,False,https://www.yelp.com/biz/sotto-mare-san-franci...,4522,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 37.79979, 'longitude': -122.40834}","[delivery, pickup]",$$,"{'address1': '552 Green St', 'address2': '', '...",+14153983181,(415) 398-3181,421.499100
1,K9XVDlPNhrrSVEJN7uWqJQ,the-italian-homemade-company-san-francisco-8,The Italian Homemade Company,https://s3-media3.fl.yelpcdn.com/bphoto/Xl1A2w...,False,https://www.yelp.com/biz/the-italian-homemade-...,1759,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.801752, 'longitude': -122.411851}","[delivery, pickup]",$$,"{'address1': '716 Columbus Ave', 'address2': '...",+14157128874,(415) 712-8874,218.158166
2,I8T8aoAsFQYwQiRvLBayFA,il-casaro-pizzeria-and-mozzarella-bar-north-be...,Il Casaro Pizzeria & Mozzarella Bar - North Beach,https://s3-media4.fl.yelpcdn.com/bphoto/ef3tF8...,False,https://www.yelp.com/biz/il-casaro-pizzeria-an...,1517,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 37.79852, 'longitude': -122.40739}","[restaurant_reservation, delivery, pickup]",$$,"{'address1': '348 Columbus Ave', 'address2': '...",+14156779455,(415) 677-9455,580.994463
3,o43B4DnnQbvkdDK6AVafQg,piccolo-forno-san-francisco,Piccolo Forno,https://s3-media2.fl.yelpcdn.com/bphoto/f5cIsI...,False,https://www.yelp.com/biz/piccolo-forno-san-fra...,792,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 37.8013699308642, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '725 Columbus Ave', 'address2': '...",+14157570087,(415) 757-0087,258.085989
4,bC3FfFqfaQu7gZznjkmt9Q,mona-lisa-restaurant-san-francisco,Mona Lisa Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/LLD2tS...,False,https://www.yelp.com/biz/mona-lisa-restaurant-...,2107,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 37.7983902, 'longitude': -122.407...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '353 Columbus Ave', 'address2': '...",+14159894917,(415) 989-4917,576.345582


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
452,TyYg1zXxGgI8X_XOfpclHw,papa-johns-pizza-san-francisco-2,Papa Johns Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/gq2_Of...,False,https://www.yelp.com/biz/papa-johns-pizza-san-...,137,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 37.78807, 'longitude': -122.416298}","[delivery, pickup]",$,"{'address1': '969 Sutter St.', 'address2': '',...",+14154407272,(415) 440-7272,1759.396186
453,xd4HRRVilNNB45Gm974XiA,dominos-pizza-san-francisco-5,Domino's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/CCGa6D...,False,https://www.yelp.com/biz/dominos-pizza-san-fra...,399,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 37.7862196415663, 'longitude': -1...",[],$,"{'address1': '876 Geary St', 'address2': '', '...",+14157760400,(415) 776-0400,1990.991551
454,NZdXTfrfCfl9nk4ZvoWT_A,lees-deli-san-francisco-7,Lee's Deli,https://s3-media4.fl.yelpcdn.com/bphoto/JxPMoF...,False,https://www.yelp.com/biz/lees-deli-san-francis...,213,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 37.785371, 'longitude': -122.395898}","[delivery, pickup]",$,"{'address1': '303 Second St', 'address2': '', ...",+14152847303,(415) 284-7303,2346.939983
455,5gXTCsfRQjVL1JO4rPVLOw,pizza-by-the-slice-san-francisco,Pizza By The Slice,https://s3-media3.fl.yelpcdn.com/bphoto/qVFeLr...,False,https://www.yelp.com/biz/pizza-by-the-slice-sa...,103,"[{'alias': 'gelato', 'title': 'Gelato'}, {'ali...",2.5,"{'latitude': 37.787134, 'longitude': -122.410153}",[],$,"{'address1': '444 Geary Blvd', 'address2': '',...",,,1787.050844
456,xsG1SSVsm0YdL6-PaoKV_A,111-minna-gallery-san-francisco,111 Minna Gallery,https://s3-media3.fl.yelpcdn.com/bphoto/pTzmdT...,False,https://www.yelp.com/biz/111-minna-gallery-san...,762,"[{'alias': 'galleries', 'title': 'Art Gallerie...",3.5,"{'latitude': 37.787336818346176, 'longitude': ...",[],$$,"{'address1': '111 Minna St', 'address2': '', '...",+14159741719,(415) 974-1719,1998.743368


In [16]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()



0

In [17]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_italian.csv.gz', compression='gzip',index=False)

